In [1]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
sys.path.insert(0, '../..')
from JKBio import TerraFunction as terra
%load_ext autoreload
from JKBio import Helper as h
%autoreload 2
import pickle
%load_ext rpy2.ipython
#%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()
import numpy as np
import itertools

from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import gseapy
from JKBio.helper import pyDESeq2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import AgglomerativeClustering

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

Loading BokehJS ...

In [8]:
! gsutil -m mv gs://transfer-amlproject/*.fastq.gz gs://transfer-amlproject/RNPv3/

Copying gs://transfer-amlproject/20200304_10_MP7624_S10_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_10_MP7624_S10_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_11_MP7624_S11_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_12_MP7624_S12_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_11_MP7624_S11_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_13_MP7624_S13_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_12_MP7624_S12_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_13_MP7624_S13_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_14_MP7624_S14_R1_001.fastq.gz [Content

Copying gs://transfer-amlproject/20200304_35_MP7624_S35_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_36_MP7624_S36_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200304_16_MP7624_S16_R2_001.fastq.gz...     
Removing gs://transfer-amlproject/20200304_14_MP7624_S14_R2_001.fastq.gz...
Removing gs://transfer-amlproject/20200304_25_MP7624_S25_R1_001.fastq.gz...     
Copying gs://transfer-amlproject/20200304_37_MP7624_S37_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_38_MP7624_S38_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_37_MP7624_S37_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200304_13_MP7624_S13_R2_001.fastq.gz...
Copying gs://transfer-amlproject/20200304_36_MP7624_S36_R2_001.fastq.gz [Content-Type=application/octet-stream]

Removing gs://transfer-amlproject/20200304_47_MP7624_S47_R2_001.fastq.gz...     
Removing gs://transfer-amlproject/20200304_50_MP7624_S50_R2_001.fastq.gz...     
Copying gs://transfer-amlproject/20200304_57_MP7624_S57_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200304_50_MP7624_S50_R1_001.fastq.gz...     
Copying gs://transfer-amlproject/20200304_57_MP7624_S57_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_58_MP7624_S58_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/20200304_58_MP7624_S58_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200304_45_MP7624_S45_R1_001.fastq.gz...
Removing gs://transfer-amlproject/20200304_44_MP7624_S44_R1_001.fastq.gz...     
Copying gs://transfer-amlproject/20200304_59_MP7624_S59_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-aml

Copying gs://transfer-amlproject/20200304_9_MP7624_S9_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://transfer-amlproject/20200304_64_MP7624_S64_R1_001.fastq.gz...     
Removing gs://transfer-amlproject/20200304_68_MP7624_S68_R1_001.fastq.gz...     
Removing gs://transfer-amlproject/20200304_69_MP7624_S69_R2_001.fastq.gz...     
Removing gs://transfer-amlproject/20200304_69_MP7624_S69_R1_001.fastq.gz...
Removing gs://transfer-amlproject/20200304_65_MP7624_S65_R2_001.fastq.gz...
Removing gs://transfer-amlproject/20200304_5_MP7624_S5_R2_001.fastq.gz...       
Removing gs://transfer-amlproject/20200304_71_MP7624_S71_R1_001.fastq.gz...     
Removing gs://transfer-amlproject/20200304_67_MP7624_S67_R2_001.fastq.gz...
Removing gs://transfer-amlproject/20200304_64_MP7624_S64_R2_001.fastq.gz...     
Removing gs://transfer-amlproject/20200304_71_MP7624_S71_R2_001.fastq.gz...     
Removing gs://transfer-amlproject/20200304_6_MP7624_S6_R2_001.fastq.gz...
Removing gs://tran

In [42]:
sampleset='MAX_AML_RNP'

In [16]:
terra.uploadFromFolder('transfer-amlproject','RNPv3/',
                       'broad-firecloud-ccle/hg38_RNAseq',samplesetname=sampleset,
                      fformat="fastqR1R2",loc=1)

please be sure you gave access to your terra email account access to this bucket
['RNPv3/20200304_10_MP7624_S10_R1_001.fastq.gz', 'RNPv3/20200304_10_MP7624_S10_R2_001.fastq.gz', 'RNPv3/20200304_11_MP7624_S11_R1_001.fastq.gz', 'RNPv3/20200304_11_MP7624_S11_R2_001.fastq.gz', 'RNPv3/20200304_12_MP7624_S12_R1_001.fastq.gz', 'RNPv3/20200304_12_MP7624_S12_R2_001.fastq.gz', 'RNPv3/20200304_13_MP7624_S13_R1_001.fastq.gz', 'RNPv3/20200304_13_MP7624_S13_R2_001.fastq.gz', 'RNPv3/20200304_14_MP7624_S14_R1_001.fastq.gz', 'RNPv3/20200304_14_MP7624_S14_R2_001.fastq.gz', 'RNPv3/20200304_15_MP7624_S15_R1_001.fastq.gz', 'RNPv3/20200304_15_MP7624_S15_R2_001.fastq.gz', 'RNPv3/20200304_16_MP7624_S16_R1_001.fastq.gz', 'RNPv3/20200304_16_MP7624_S16_R2_001.fastq.gz', 'RNPv3/20200304_17_MP7624_S17_R1_001.fastq.gz', 'RNPv3/20200304_17_MP7624_S17_R2_001.fastq.gz', 'RNPv3/20200304_18_MP7624_S18_R1_001.fastq.gz', 'RNPv3/20200304_18_MP7624_S18_R2_001.fastq.gz', 'RNPv3/20200304_19_MP7624_S19_R1_001.fastq.gz', 'RNPv3

[73 rows x 3 columns]
> /home/jeremie/JKBio/TerraFunction.py(230)uploadFromFolder()
    229     print(df)
--> 230     df["Source"] = source
    231     df["participant"] = data['sample_id']

ipdb> c
Successfully imported 73 participants.
Successfully imported 73 samples.
Successfully imported 1 sample sets:
  * RNPv3 (73 samples)


In [6]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/hg38_RNAseq')

In [18]:
submission_id = wm.create_submission("star_v1-0_BETA_cfg", sampleset, 'sample_set',expression='this.samples')

Successfully created submission 8043053e-0bd6-4a55-a23e-6d4fe9b0991b.


In [7]:
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

1.0 of jobs Succeeded in submission 0.sion 0. 224 mn elapsed..


[]

In [8]:
submission_id = wm.create_submission("rsem_v1-0_BETA_cfg", 
                                      sampleset,'sample_set',expression='this.samples')
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

Successfully created submission 4450c5aa-6f67-4068-91ce-55a156c58342.
1.0 of jobs Succeeded in submission 0.sion 0. 560 mn elapsed..


[]

In [9]:
submission_id = wm.create_submission("rsem_aggregate_results_v1-0_BETA_cfg", 
                                         samplesetname)
terra.waitForSubmission('broad-firecloud-ccle/hg38_RNAseq', submission_id)

NameError: name 'samplesetname' is not defined

In [43]:
results = wm.get_sample_sets().loc[sampleset]
rsem_genes_expected_count = results['rsem_genes_expected_count']

In [44]:
results

samples                            [BRD4-r1, BRD4-r2, BRD4-r3, CDK6-r1, CDK6-r2, ...
rsem_transcripts_isopct            gs://fc-secure-163bcce1-14a1-4cc2-b8f8-ec8bcba...
rsem_transcripts_tpm               gs://fc-secure-163bcce1-14a1-4cc2-b8f8-ec8bcba...
rsem_transcripts_expected_count    gs://fc-secure-163bcce1-14a1-4cc2-b8f8-ec8bcba...
rsem_genes_tpm                     gs://fc-secure-163bcce1-14a1-4cc2-b8f8-ec8bcba...
rsem_genes_expected_count          gs://fc-secure-163bcce1-14a1-4cc2-b8f8-ec8bcba...
Name: MAX_AML_RNP, dtype: object

In [45]:
! gsutil cp $rsem_genes_expected_count ../data/

Copying gs://fc-secure-163bcce1-14a1-4cc2-b8f8-ec8bcbabe2da/7fcfa027-1093-456b-9811-c19905978cff/rsem_aggregate_results_workflow/18cf5a35-3707-482f-bb5a-bd295fdf30aa/call-rsem_aggregate_results/MAX_AML_2.rsem_genes_expected_count.txt.gz...
/ [1 files][  3.9 MiB/  3.9 MiB]                                                
Operation completed over 1 objects/3.9 MiB.                                      


In [46]:
file = '../data/'+rsem_genes_expected_count.split('/')[-1]

In [47]:
file

'../data/MAX_AML_2.rsem_genes_expected_count.txt.gz'

In [48]:
! gunzip $file

In [49]:
rsem_genes_expected_count = pd.read_csv(file[:-3], sep='\t')

In [82]:
data = rsem_genes_expected_count.drop("transcript_id(s)",1)

In [83]:
data["gene_id"] = h.convertGenes(data['gene_id'])[0]

you need access to taiga for this (https://pypi.org/project/taigapy/)
20702 could not be parsed... we don't have all genes already


In [84]:
data=data.set_index('gene_id')

In [85]:
experiments = set([i.split('-')[0] for i in data.columns])
experiments

{'BRD4',
 'CDK6',
 'CEBPa',
 'ELF2',
 'FLI1',
 'GFI1',
 'IKZF1',
 'IRF2BP2',
 'IRF8',
 'LMO2',
 'LYL1',
 'MAX',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MYB',
 'MYC',
 'Non',
 'RUNX1',
 'RUNX2',
 'SPI1',
 'ZEB2',
 'ZMYND8'}

## pre processing 

filter some more

In [86]:
toremove = np.argwhere(data.values.var(1)==0)
toremove.ravel()

array([   24,    70,   127, ..., 58810, 58811, 58812])

In [87]:
toremove.shape

(19537, 1)

In [88]:
data = data.drop(data.iloc[toremove.ravel()].index,0)

In [89]:
data.shape

(39240, 72)

renormalize the data

## Getting the Core TF information

In [33]:
ctf=pd.read_csv('../data/CTF.csv',header=None)[0].values.tolist()
ctf

['ARID2',
 'CEBPA',
 'CEBPE',
 'E2F3',
 'FLI1',
 'FOSL2',
 'GFI1',
 'GFI1B',
 'HHEX',
 'IRF8',
 'LYL1',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MTF1',
 'MYB',
 'MYC',
 'PLAGL2',
 'RUNX1',
 'RUNX2',
 'RXRA',
 'SETDB1',
 'SNAPC5',
 'SP1',
 'SPI1',
 'SREBF1',
 'STAT5B',
 'TERF2',
 'TFAP4',
 'ZEB2',
 'ZFPM1',
 'ZMYND8',
 'LMO2',
 'MAX',
 'ELF2',
 'ETV6',
 'HOXA9',
 'GATA2']

In [34]:
genenames = data.index
ctfpos = [i for i, val in enumerate(genenames) if val in ctf]
notctfpos = [i for i, val in enumerate(genenames) if val not in ctf]

We find a CTF not in the dataset

In [ ]:
[val for val in ctf if val not in genenames]

## DESEQ ANALYSIS

In [90]:
expe = {}
for val in experiments: 
    expe[val] = [1 if val in col else 0 for col in data.columns]

In [65]:
data = data.reset_index()

In [67]:
experiments.remove('Non')
results = {}

KeyError: 'Non'

In [62]:
experiments - set(ctf)

{'BRD4', 'CDK6', 'CEBPa', 'IKZF1', 'IRF2BP2'}

In [63]:
experiments

{'BRD4',
 'CDK6',
 'CEBPa',
 'ELF2',
 'FLI1',
 'GFI1',
 'IKZF1',
 'IRF2BP2',
 'IRF8',
 'LMO2',
 'LYL1',
 'MAX',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MYB',
 'MYC',
 'RUNX1',
 'RUNX2',
 'SPI1',
 'ZEB2',
 'ZMYND8'}

In [66]:
for val in experiments:  
    design = pd.DataFrame(index=data.columns[1:], columns=['DMSO','Target'], 
                          data=np.array([expe['Non'],expe[val]]).T)
    design.index = design.index.astype(str).str.replace('-','.')
    deseq = pyDESeq2.pyDESeq2(count_matrix=data, design_matrix = design, 
                              design_formula='~DMSO + Target', gene_column="gene_id")
    deseq.run_deseq()
    deseq.get_deseq_result()
    r = deseq.deseq_result
    r.pvalue = np.nan_to_num(np.array(r.pvalue), 1)
    r.log2FoldChange = np.nan_to_num(np.array(r.log2FoldChange), 0)
    results[val] = r

KeyError: 'Non'

## what are most downregulated (volcano)

In [ ]:
MS2res.pvalue = np.nan_to_num(np.array(MS2res.pvalue), 1)
MS2res.log2FoldChange = np.nan_to_num(np.array(MS2res.log2FoldChange), 0)
MS2res.gene_id = convertGenes(MS2res.gene_id)[0]
show(volcano(MS2res,tohighlight=ctf))

## CTF (volcano)

In [ ]:
results

In [ ]:
for val in experiments:
    a = volcano(results[val],tohighlight=ctf,title=val)
    try:
        show(a)
    except RuntimeError:
        show(a)

In [ ]:
data = data.set_index('gene_id')

## any bias in the data

In [ ]:
red = PCA(50).fit_transform(data.T)
a = scatter(red, labels=data.columns)

In [ ]:
red.shape

In [ ]:
red = TSNE(2,10).fit_transform(red)

In [ ]:
a = scatter(red[:,:2], labels=data.columns, radi=10)

In [ ]:
show(a)

In [ ]:
dataclean = data[data.max(1) > 10].drop(['LMO2-r1','LMO2-r2','LMO2-r3'],1)

In [ ]:
for val in grouped(dataclean.columns,3):
    val = list(val)
    dataclean[val[0].split('-')[0]] = dataclean[val].mean(1)
    dataclean = dataclean.drop(val,1)

In [ ]:
pca = PCA(20)
red = pca.fit_transform(dataclean.T)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
a = scatter(red[:,:2], labels=dataclean.columns, radi=4000, title='batch effect RNP')

MAX CEBPa ZEB2 

In [ ]:
show(a)

In [ ]:
save(a,'data/plots/batcheffect_RNP.html')

## GSEA analysis

In [68]:
res = {}

In [70]:
totest

,Non-target-r1,Non-target-r2,Non-target-r3,Non-target-r4,Non-target-r5,Non-target-r6,Non-target-r7,ZEB2-r1,ZEB2-r2,ZEB2-r3
0,0.00,0.00,1.00,0.00,1.00,0.00,0.00,4.00,2.0,1.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
2,695.00,629.00,669.00,976.00,1035.00,1044.00,898.00,1107.00,758.0,929.00
3,189.23,134.64,152.49,209.93,217.91,254.92,227.88,305.21,241.5,276.69
4,352.77,292.36,324.51,500.07,526.09,524.08,272.12,433.79,295.5,335.31
...,...,...,...,...,...,...,...,...,...,...
39235,0.00,6.28,2.41,16.38,9.16,20.02,11.50,0.00,10.9,5.28
39236,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
39237,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
39238,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.0,0.00


In [71]:
cls

['DMSO',
 'DMSO',
 'DMSO',
 'DMSO',
 'DMSO',
 'DMSO',
 'DMSO',
 'Condition',
 'Condition',
 'Condition']

In [72]:
res[val]

KeyError: 'ZEB2'

In [79]:
data['gene_id'] = data.index

In [81]:
data

,gene_id,BRD4-r1,BRD4-r2,BRD4-r3,CDK6-r1,CDK6-r2,CDK6-r3,CEBPa-r1,CEBPa-r2,CEBPa-r3,...,RUNX2-r3,SPI1-r1,SPI1-r2,SPI1-r3,ZEB2-r1,ZEB2-r2,ZEB2-r3,ZMYND8-r1,ZMYND8-r2,ZMYND8-r3
0,0,0.00,2.00,0.00,1.00,0.00,3.00,0.00,0.00,0.00,...,0.00,2.00,1.00,1.00,4.00,2.0,1.00,3.00,1.00,1.00
1,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
2,2,543.00,646.00,700.00,738.00,645.00,687.00,1079.00,1050.00,1149.00,...,873.00,903.00,749.00,701.00,1107.00,758.0,929.00,521.00,715.00,688.00
3,3,158.33,191.89,158.28,205.39,161.19,219.88,303.40,346.63,327.12,...,207.90,227.20,232.98,224.73,305.21,241.5,276.69,152.57,177.88,206.75
4,4,260.67,250.11,293.72,332.61,262.81,242.12,549.60,502.37,571.88,...,358.10,417.80,354.02,315.27,433.79,295.5,335.31,265.43,375.12,317.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39235,39235,4.68,9.67,2.86,5.29,6.11,8.85,8.72,12.55,22.22,...,2.67,19.58,7.44,6.67,0.00,10.9,5.28,4.94,5.68,2.73
39236,39236,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
39237,39237,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00
39238,39238,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00


In [80]:
for val in experiments:
    print(val)
    totest = data[[v for v in data.columns if val in v or 'Non' in v]]
    cls = ['Condition' if val in v else 'DMSO' for v in totest.columns]
    res[val] = gseapy.gsea(data=totest, gene_sets='WikiPathways_2013', 
                cls= cls, no_plot=False, processes=8)
    res[val].res2d['Term'] = [i.split('_')[0][:50] for i in res[val].res2d.index]
    
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)

ZEB2


2020-06-05 18:18:51,134 Warning: dropping duplicated gene names, only keep the first values
/home/jeremie/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
2020-06-05 18:18:55,446 No gene sets passed through filtering condition!!!, try new parameters again!
Note: check gene name, gmt file format, or filtering size.


SystemExit: 0

/home/jeremie/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
with open('../data/wikipathway_RNPv1', 'wb') as f:
    pickle.dump(res,f)

In [74]:
with open('../data/wikipathway_RNPv1','rb') as f:
    res = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'wikipathway'

In [ ]:
for i, val in enumerate(experiments):
    plt.figure(i)
    res[val].res2d.Term = [' '.join(i.split('_')[1:-2]) for i in res[val].res2d.index]
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)

In [ ]:
a = set()
for k, val in res.items():
    a.update(set(val.res2d.index))
a = {i:[0]*len(res) for i in a}
for n,(k, val) in enumerate(res.items()):
    for i,v in val.res2d.iterrows():
        a[i][n] = v.es
res = pd.DataFrame(a, index=res.keys())
fig, ax = plt.subplots(figsize=(20,15))
sns.heatmap(ax=ax,data=res)

In [ ]:
model = AgglomerativeClustering(n_clusters=6,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(res)
ii = itertools.count(res.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()
a = plotCorrelationMatrix(res.values[sort],res.index[sort].tolist(),interactive=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))
sns.heatmap(ax=ax,data=res)

In [ ]:
fig.savefig("enriched_terms.png")

In [ ]:
show(a)

In [91]:
experiments

{'BRD4',
 'CDK6',
 'CEBPa',
 'ELF2',
 'FLI1',
 'GFI1',
 'IKZF1',
 'IRF2BP2',
 'IRF8',
 'LMO2',
 'LYL1',
 'MAX',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MYB',
 'MYC',
 'Non',
 'RUNX1',
 'RUNX2',
 'SPI1',
 'ZEB2',
 'ZMYND8'}

In [94]:
data

,BRD4-r1,BRD4-r2,BRD4-r3,CDK6-r1,CDK6-r2,CDK6-r3,CEBPa-r1,CEBPa-r2,CEBPa-r3,ELF2-r1,...,SPI1-r1,SPI1-r2,SPI1-r3,ZEB2-r1,ZEB2-r2,ZEB2-r3,ZMYND8-r1,ZMYND8-r2,ZMYND8-r3,gene_id
gene_id,,,,,,,,,,,,,,,,,,,,,
TSPAN6,0.00,2.00,0.00,1.00,0.00,3.00,0.00,0.00,0.00,0.00,...,2.00,1.00,1.00,4.00,2.0,1.00,3.00,1.00,1.00,TSPAN6
TNMD,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,TNMD
DPM1,543.00,646.00,700.00,738.00,645.00,687.00,1079.00,1050.00,1149.00,613.00,...,903.00,749.00,701.00,1107.00,758.0,929.00,521.00,715.00,688.00,DPM1
SCYL3,158.33,191.89,158.28,205.39,161.19,219.88,303.40,346.63,327.12,195.05,...,227.20,232.98,224.73,305.21,241.5,276.69,152.57,177.88,206.75,SCYL3
C1orf112,260.67,250.11,293.72,332.61,262.81,242.12,549.60,502.37,571.88,286.95,...,417.80,354.02,315.27,433.79,295.5,335.31,265.43,375.12,317.25,C1orf112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285991,4.68,9.67,2.86,5.29,6.11,8.85,8.72,12.55,22.22,9.39,...,19.58,7.44,6.67,0.00,10.9,5.28,4.94,5.68,2.73,ENSG00000285991
ENSG00000285992,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,ENSG00000285992
ENSG00000285993,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,ENSG00000285993


In [95]:
for i, val in enumerate(experiments):
    print(val)
    totest = data[[v for v in data.columns if val in v or 'Non' in v]]
    cls = ['Condition' if val in v else 'DMSO' for v in totest.columns]
    res[val] = gseapy.gsea(data=totest, gene_sets='GO_Biological_Process_2015', 
                cls= cls, no_plot=False, processes=6)
    res[val].res2d['Term'] = [i.split('_')[0][:50] for i in res[val].res2d.index]
    plt.figure(i)
    sns.barplot(data=res[val].res2d.iloc[:25], x="es", y="Term",
                hue_order="geneset_size").set_title(val)

ZEB2
Non


IndexError: list index out of range

In [ ]:
with open('GO_Biological_Process_2015', 'wb') as f:
    pickle.dump(res,f)

In [ ]:
with open('GO_Biological_Process_2015','rb') as f:
    res = pickle.load(f)

creating matrices

In [ ]:
a = set()
for k, val in res.items():
    a.update(set(val.res2d.Term))
a = {i:[0]*len(res) for i in a}
for n,(k, val) in enumerate(res.items()):
    for i,v in val.res2d.iterrows():
        a[v.Term][n] = v.es
res = pd.DataFrame(a, index=res.keys())
fig, ax = plt.subplots(figsize=(20,15))
sns.heatmap(ax=ax,data=res)

In [ ]:
model = AgglomerativeClustering(n_clusters=5,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(res)
ii = itertools.count(res.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]

In [ ]:
sort = labels.argsort()

In [ ]:
a = plotCorrelationMatrix(res.values[sort],res.index[sort].tolist(),interactive=True)

In [ ]:
show(a)

In [ ]:
cluster1= ['LMO2','LYL1','MAX','MEF2C']
cluster2=['GFI1','FLI1','MYB','IKZF1','ELF2','CEBPa','MEIS1']
cluster3=['IRF2BP2','MEF2C','CDK6','MEF2D','IRF8','BRD4','MYC']
cluster4= ['RUNX1','RUNX2','ZMYND8']

In [ ]:
res.loc[cluster2].mean().sort_values()

In [ ]:
'GO_Molecular_Function_2015',
'GeneSigDB',
'ENCODE_TF_ChIP-seq_2014',
#'Drug_Perturbations_from_GEO_2014',
'GO_Cellular_Component_2015',
'GO_Biological_Process_2015',
'PPI_Hub_Proteins',
'WikiPathways_2013',
'TF-LOF_Expression_from_GEO',
# msig db C2 C6 H http://software.broadinstitute.org/gsea/msigdb/annotate.jsp
# max's crc

## compare to the drop of CTF

In [ ]:
ctf = [
 'BRD4',
 'CDK6',
 'CEBPA',
 'ELF2',
 'FLI1',
 'GFI1',
 'IKZF1',
 'IRF2BP2',
 'IRF8',
 'LMO2',
 'LYL1',
 'MAX',
 'MEF2C',
 'MEF2D',
 'MEIS1',
 'MYB',
 'MYC',
 'RUNX1',
 'RUNX2',
 'SPI1',
 'ZEB2',
 'ZMYND8'
]

In [ ]:
deseq = pd.DataFrame()
for k, val in results.items():
    deseq[k] = val.log2FoldChange
deseq=deseq.T

In [ ]:
deseq

In [ ]:
a = plotCorrelationMatrix(a,
                          deseq.index[sort].tolist(),interactive=True)

In [ ]:
ctf[11] = 'CEBPa'
ctf[]

In [ ]:
ctf

dropping ETV6 SP1  GSE1 LDB1

In [ ]:
deseq.loc[['MYC',
 'MYB',
 'SPI1',
 'RUNX1',
 'IRF2BP2',
 'FLI1',
 'ELF2',
 'ZEB2',
 'GFI1',
 'LMO2',
 'CEBPa',
 'MEF2D',
 'MEF2C',
 'IRF8',
 'MEIS1',
 'RUNX2',
 'RUNX2',
 'ZMYND8']]

In [ ]:
show(a)

In [ ]:
deseq_ctf = deseq.loc[['MYC',
 'MYB',
 'SPI1',
 'RUNX1',
 'IRF2BP2',
 'FLI1',
 'ELF2',
 'ZEB2',
 'GFI1',
 'LMO2',
 'CEBPa',
 'MEF2D',
 'MEF2C',
 'IRF8',
 'MEIS1',
 'RUNX2',
 'ZMYND8']]
model = AgglomerativeClustering(n_clusters=7,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(deseq_ctf)
ii = itertools.count(deseq_ctf.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()
a = deseq_ctf.values[sort]
a = plotCorrelationMatrix(a, deseq_ctf.index[sort].tolist(),interactive=True)

In [ ]:
show(a)

In [ ]:
model = AgglomerativeClustering(n_clusters=7,linkage="average", 
                                affinity="cosine", compute_full_tree=True)
labels = model.fit_predict(deseq)
ii = itertools.count(deseq.shape[0])
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in model.children_]
sort = labels.argsort()
a = deseq.values[sort]
a = plotCorrelationMatrix(a, deseq.index[sort].tolist(),interactive=True)

In [ ]:
show(a)

tsne, pca, clustering accross TF, CRC, most var genes, both ways.